Testing CNN method from https://wandb.ai/mostafaibrahim17/ml-articles/reports/An-Introduction-to-Audio-Classification-with-Keras--Vmlldzo0MDQzNDUy#:~:text=%EF%BB%BFKeras%20is%20a%20go,both%20beginners%20and%20advanced%20users.

https://keras.io/guides/sequential_model/

In [108]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras import layers # example functions: Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
from keras import Input
import matplotlib.pyplot as plt
import time

In [109]:
def get_data(audio_path, meta_file, label_name):
    songs_matrix = []
    songs_label = []
    audio_list = os.listdir(audio_path)
    df = pd.read_csv(meta_file)
    for name in audio_list:
        #name = audio_list[0]
        number = name.replace('_', '.').split('.')[0]

        song, sr = librosa.load(os.path.join(audio_path, name))
        mfccs = librosa.feature.mfcc(y = song, n_mfcc = 60)
        #mfcc = np.mean(mfccs, axis=1) # average over columns, shape(n_mfcc, )

        songs_matrix.append(mfccs)
        songs_label.append(df[label_name].loc[df.cat_num == float(number)].item())
    return np.array(songs_matrix), np.array(songs_label)

In [110]:
# load all audio in audio_path
# convert to matrix, save in np array, 
# load the corresponding label_name from lable_filename, save in another np array

audio_path = 'E:\chirpfiles\selected_files'
meta_file = 'MLNS_Insects_Fams_05212024.csv'
label_name = 'critter_name'
start = time.time()
songs_matrix, songs_label = get_data(audio_path, meta_file, label_name)
end = time.time()
print('time loading:', end - start)

time loading: 31.243870496749878


In [105]:
print(np.unique(songs_label, return_counts=True))

(array(['cicada', 'cricket', 'grasshopper', 'kaydid'], dtype='<U11'), array([ 25, 389,   2, 649], dtype=int64))


In [111]:
# convert insec fam into numbers. use le.inverse_transform to go from number to name
le = LabelEncoder()
songs_label_num = le.fit_transform(songs_label)
songs_label_onehot = to_categorical(songs_label_num) # why do we do this??

In [112]:
#np.unique(songs_label, return_counts=True)
#print(songs_matrix.shape)
sub_songs_matrix = songs_matrix[:, :, 0:600]

if ever need to convert onehot matrix back to string label
#label_num = np.sum(songs_label_onehot*np.arange(0,songs_label_onehot.shape[1]), axis=1)
#print(le.inverse_transform(label_num.astype(int)))

In [113]:
X_train, X_test, y_train, y_test = train_test_split(sub_songs_matrix, songs_label_onehot, 
                                                    test_size = .2, random_state = 404)
## from bootcamp notes
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],X_train.shape[2],1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],1))

X_train.shape

(852, 60, 600, 1)

In [122]:
#input_shape = (X_train.shape[1], 1) # n_featuer-by-1 vector
input_shape =  (X_train.shape[1], X_train.shape[2],1,)

model = Sequential() 
model.add(Input(input_shape))
model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2,2), strides=2))
#model.add(layers.Dropout(0.2))
#model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
#model.add(layers.MaxPooling2D((2,2), strides=2))
#model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(len(le.classes_), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()

In [126]:
start = time.time()
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
end = time.time()
print('fit time: ', end - start)
#predicted_vector = model.predict(X_test)
#predicted_class_index = np.argmax(predicted_vector, axis=-1)
#le.inverse_transform(predicted_class_index)


Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 560ms/step - accuracy: 0.9871 - loss: 0.3864 - val_accuracy: 0.8638 - val_loss: 12.9327
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 551ms/step - accuracy: 0.9951 - loss: 0.3456 - val_accuracy: 0.8779 - val_loss: 7.5852
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 565ms/step - accuracy: 0.9569 - loss: 1.2675 - val_accuracy: 0.8732 - val_loss: 7.4365
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 557ms/step - accuracy: 0.9651 - loss: 1.5364 - val_accuracy: 0.8545 - val_loss: 7.2133
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 566ms/step - accuracy: 0.9933 - loss: 0.1675 - val_accuracy: 0.8592 - val_loss: 8.9742
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 552ms/step - accuracy: 0.9893 - loss: 0.3798 - val_accuracy: 0.8545 - val_loss: 10.8310
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 551ms/step - accuracy: 0.9914 - loss: 0.1974 - val_accuracy: 0.8122 - val_loss: 12.2783
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 569ms/step - accuracy: 0.9905 - loss: 0.6680 - val_a

In [127]:
predicted_vector = model.predict(X_test)
predicted_class_index = np.argmax(predicted_vector, axis=-1)
le.inverse_transform(predicted_class_index)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step


array(['cricket', 'kaydid', 'kaydid', 'cricket', 'cricket', 'cricket',
       'cricket', 'cricket', 'kaydid', 'kaydid', 'cricket', 'kaydid',
       'kaydid', 'cricket', 'kaydid', 'kaydid', 'cricket', 'cricket',
       'kaydid', 'cricket', 'cricket', 'kaydid', 'cricket', 'cricket',
       'kaydid', 'kaydid', 'kaydid', 'kaydid', 'kaydid', 'cricket',
       'cricket', 'kaydid', 'kaydid', 'kaydid', 'cricket', 'cricket',
       'kaydid', 'cricket', 'cricket', 'cricket', 'cricket', 'cricket',
       'cricket', 'cricket', 'cricket', 'kaydid', 'cricket', 'cricket',
       'cricket', 'cricket', 'kaydid', 'kaydid', 'cricket', 'cricket',
       'kaydid', 'cricket', 'cricket', 'kaydid', 'cricket', 'kaydid',
       'kaydid', 'kaydid', 'kaydid', 'cricket', 'cricket', 'kaydid',
       'kaydid', 'cricket', 'cricket', 'kaydid', 'cricket', 'kaydid',
       'cricket', 'kaydid', 'kaydid', 'cricket', 'cricket', 'kaydid',
       'kaydid', 'kaydid', 'kaydid', 'cricket', 'kaydid', 'cricket',
       'cricket',

Does training repeatedly increase performance??
What does Epoch and batch size do?

using conv2d(64), maxpool, flatten, dense, dense, batch size 32
Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 552ms/step - accuracy: 0.9391 - loss: 6.2835 - val_accuracy: 0.8545 - val_loss: 11.0454
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 549ms/step - accuracy: 0.9566 - loss: 8.4944 - val_accuracy: 0.8216 - val_loss: 11.1535
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 560ms/step - accuracy: 0.8765 - loss: 8.8313 - val_accuracy: 0.7230 - val_loss: 23.5377
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 565ms/step - accuracy: 0.9043 - loss: 6.7910 - val_accuracy: 0.8075 - val_loss: 14.2637
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 16s 580ms/step - accuracy: 0.9250 - loss: 6.7538 - val_accuracy: 0.7653 - val_loss: 15.3153
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 559ms/step - accuracy: 0.9108 - loss: 6.5910 - val_accuracy: 0.8920 - val_loss: 6.9638
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 551ms/step - accuracy: 0.9831 - loss: 1.6024 - val_accuracy: 0.8310 - val_loss: 12.2536
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 562ms/step - accuracy: 0.9745 - loss: 1.0641 - val_accuracy: 0.8498 - val_loss: 11.0394
Epoch 9/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 551ms/step - accuracy: 0.9707 - loss: 0.9651 - val_accuracy: 0.8216 - val_loss: 14.1250
Epoch 10/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 557ms/step - accuracy: 0.9534 - loss: 1.5526 - val_accuracy: 0.8451 - val_loss: 7.3341
fit time:  151.07773733139038

In [128]:
y_test = y_test.astype(int) 
y_test_cate = list(y_test[:,1] + 2*y_test[:,2]+3*y_test[:,3]) 
y_test_names = le.inverse_transform(y_test[:,1] + 2*y_test[:,2]+3*y_test[:,3])
print(y_test_names)

['cricket' 'kaydid' 'kaydid' 'cricket' 'kaydid' 'cricket' 'cricket'
 'cricket' 'cricket' 'cicada' 'cicada' 'kaydid' 'kaydid' 'cricket'
 'kaydid' 'kaydid' 'cricket' 'kaydid' 'kaydid' 'kaydid' 'cricket' 'kaydid'
 'kaydid' 'kaydid' 'kaydid' 'kaydid' 'cricket' 'kaydid' 'kaydid' 'cricket'
 'cricket' 'kaydid' 'kaydid' 'kaydid' 'kaydid' 'cricket' 'kaydid'
 'cricket' 'cricket' 'kaydid' 'cricket' 'kaydid' 'kaydid' 'cricket'
 'cricket' 'kaydid' 'kaydid' 'cicada' 'cricket' 'cicada' 'kaydid' 'kaydid'
 'kaydid' 'cricket' 'kaydid' 'kaydid' 'cricket' 'kaydid' 'cricket'
 'kaydid' 'kaydid' 'kaydid' 'kaydid' 'cricket' 'cricket' 'kaydid' 'kaydid'
 'cricket' 'kaydid' 'kaydid' 'cricket' 'kaydid' 'kaydid' 'kaydid' 'kaydid'
 'cricket' 'cricket' 'kaydid' 'cicada' 'kaydid' 'kaydid' 'kaydid' 'kaydid'
 'cricket' 'cricket' 'cricket' 'kaydid' 'cricket' 'cricket' 'kaydid'
 'cricket' 'kaydid' 'kaydid' 'cricket' 'kaydid' 'kaydid' 'kaydid'
 'cricket' 'kaydid' 'cricket' 'kaydid' 'cricket' 'kaydid' 'cricket'
 'kaydid' '

In [129]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test_cate, predicted_class_index)
pd.DataFrame(conf_mat)#,
#                 columns = ['Predicted 0', 'Predicted 1', 'Predicted 2', 'Predicted 3'],
#                 index = ['Actual 0', 'Actual 1', 'Actual 2', 'Actual 3'])

,0,1,2
0,0,3,3
1,0,73,3
2,0,35,96


In [75]:
print(y_test_cate)

[0, 1, 1, 3, 1, 3, 3, 0, 0, 0, 1, 0, 1, 0, 3, 1, 0, 3, 3, 0, 3, 3, 3, 1, 1, 1, 3, 3, 3, 3, 1, 3, 1, 3, 1, 1, 0, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 1, 1, 3, 1, 1, 1]
